<a href="https://colab.research.google.com/github/christinezuzart/DeepLearning/blob/master/ULMFit_ClassifierTraining_FineTuning1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi 
!pip install gputil 
!pip install psutil
!pip install humanize
import psutil 
import humanize 
import os 
import GPUtil as GPU 
GPUs = GPU.getGPUs()

# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0] 
def printm(): 
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss)) 
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal)) 

printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=adfff9a0c2c3a27a9ffc0c5fb076e23f431700c46e4d5756aeb66e9f6c3277ec
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  | Proc size: 155.9 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from fastai.text import *

In [0]:
# args also need to be passed
data_lm = load_data(F"/content/gdrive/My Drive/", 'tmp_lm.pkl',bs=32)

In [0]:
data_clas = (TextList.from_csv(F'/content/gdrive/My Drive/', 'df_classifier_clean.csv', cols='text', vocab=data_lm.vocab).split_from_df(col='is_valid').label_from_df(cols='label').databunch(bs=32))

In [6]:
classifier_learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
classifier_learn.load_encoder('/content/gdrive/My Drive/lm_fine_tuned_enc')
classifier_learn.load('/content/gdrive/My Drive/trained_model2')


RNNLearner(data=TextClasDataBunch;

Train: LabelList (160398 items)
x: TextList
xxbos " xxmaj it has no side effect , i take it in combination of xxmaj bystolic 5 xxmaj mg and xxmaj fish xxmaj oil ",xxbos " xxmaj my son is halfway through his fourth week of xxmaj intuniv . xxmaj we became concerned when he began this last week , when he started taking the highest dose he will be on . xxmaj for two days , he could hardly get out of bed , was very cranky , and slept for nearly 8 hours on a drive home from school vacation ( very unusual for him . ) i called his doctor on xxmaj monday morning and she said to stick it out a few days . xxmaj see how he did at school , and with getting up in the morning . xxmaj the last two days have been problem free . xxmaj he is xxup much more agreeable than ever . xxmaj he is less emotional ( a good thing ) , less cranky . xxmaj he is remembering all the things he should . xxmaj overall his behavior is better . 
 
  xxmaj we have tried many different medi

In [7]:
classifier_learn.unfreeze()
classifier_learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.963560,0.988995,0.785812,13:21
1,0.869471,0.965649,0.794890,13:47


In [8]:
model_save_name = 'export3.pkl'
path = F"/content/gdrive/My Drive/{model_save_name}" 

classifier_learn.export(path)
classifier_learn.save("/content/gdrive/My Drive/trained_model3", return_path=True)

PosixPath('/content/gdrive/My Drive/trained_model3.pth')

In [0]:
classifier_learn.fit_one_cycle(5, 1e-3, moms=(0.8,0.7))

In [0]:
model_save_name = 'export4.pkl'
path = F"/content/gdrive/My Drive/{model_save_name}" 

classifier_learn.export(path)
classifier_learn.save("/content/gdrive/My Drive/trained_model4", return_path=True)

In [0]:
classifier_learn.fit_one_cycle(5, 1e-3, moms=(0.8,0.7))

In [0]:
model_save_name = 'export5.pkl'
path = F"/content/gdrive/My Drive/{model_save_name}" 

classifier_learn.export(path)
classifier_learn.save("/content/gdrive/My Drive/trained_model5", return_path=True)

In [0]:
classifier_learn.show_results(rows=10)

In [0]:
from sklearn import metrics

preds, targets = classifier_learn.get_preds()
predictions = np.argmax(preds, axis=1)
confusion_matrix = metrics.confusion_matrix(targets, predictions)

filename = 'Confusion2.csv'
path = F"/content/gdrive/My Drive/{filename}" 
pd.DataFrame(confusion_matrix).to_csv(path)

In [0]:
diagonal = np.diag(confusion_matrix)
true_positives = sum(diagonal)
total_test_data = np.sum(confusion_matrix, axis=1).sum()
accuracy = (true_positives /total_test_data) * 100

print("Test Accuracy Percentage:", accuracy)